---
## Aprendizaje Supervisado (Microsoft Malware Prediction)

`Objetivo`: Estimar la probabilidad de que un equipo con Windows se infecte con malware, utilizando las propiedades del sistema.

---
## Instalacion y configuracion de librerias

In [ ]:
import importlib.util
import subprocess

def install_if_missing(package):
    if importlib.util.find_spec(package) is None:
        print(f"Instalando {package}...")
        subprocess.check_call(["pip", "install", package])
    else:
        print(f"{package} ya está instalado.")

# Lista de paquetes necesarios
required_packages = ["pandas","numpy","matplotlib","seaborn","scikit-learn", "rich"]

# Verificar e instalar los paquetes necesarios
for package in required_packages:
    install_if_missing(package)

print("Instalación de paquetes completada.")

# El paquete rich nos ayudara a mostrar de una forma mas atractiva los "print"
from rich.console import Console
console = Console()

# Y esta otra herramienta nos ayudara a visualizar mejor los datos devueltos en los "print"
from rich.table import Table

import pandas as pd
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.float_format', '{:.4f}'.format) # Esta opcion los valores float se muestren sin notacion cientifica.

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler



---
## Descarga y carga del data frame
En este paso, se descarga el archivo de datos desde un enlace proporcionado y se carga en un DataFrame de pandas para su posterior análisis.

In [ ]:
# Configura el origen de los datos según tu necesidad:

# Para leer el archivo desde una URL remota:
# url = "https://www.dropbox.com/scl/fi/uvv7j1bragzqkz9zwyvj0/sample_mmp.csv?rlkey=i0mlaxzq6e3blblfu9mhrdpsm&e=1&dl=1"
# df = pd.read_csv(url)

# Para leer el archivo desde una ubicación local:
df = pd.read_csv("sample_mmp.csv")

---
## Preprocesamiento
En esta sección, realizaremos un análisis preliminar y definiremos las variables y métodos necesarios para la limpieza y transformación de los datos. Esto incluirá el tratamiento de valores nulos, la codificación de variables y el escalado, preparando así los datos para un análisis más efectivo.

Primero haremos una exploracion basica de los datos y sacaremos conclusiones y decidiremos cuales funcions nos haran falta.

In [ ]:
df.describe().T

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum() / len(df) * 100


Tras analizar los datos y examinar las columnas individualmente en notebooks personales, organizamos su distribución de manera equitativa para que cada miembro del equipo tuviera un número exacto y balanceado de columnas:
``` python
fran_columns = list(df.columns[0:21])
ignacio_columns = list(df.columns[21:42])
marc_columns = list(df.columns[42:63])
alvaro_columns = list(df.columns[63:85])
```

Como equipo, hemos decidido implementar los siguientes métodos para gestionar los datos de forma más estructurada y metódica.

---
## funcion: `drop_unique_columns()`
Detecta y elimina columnas en las que cada valor es único (número de valores únicos igual al número de filas), ya que probablemente se trate de índices internos que no aportan información útil.

In [7]:
def drop_unique_columns(df):
    console = Console()
    total_rows = len(df)
    cols_to_drop = []

    for col in df.columns:
        if df[col].nunique() == total_rows:
            cols_to_drop.append(col)
    
    if cols_to_drop:
        console.print(f"[bold green]Columnas eliminadas por tener valores únicos en cada fila:[/bold green] {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
    else:
        console.print("[bold yellow]No se encontraron columnas con valores únicos en cada fila.[/bold yellow]")
    
    return df

---
## funcion: `classify_columns()`
Automatiza la clasificación por tipo de datos para aplicar tratamientos específicos.

In [8]:
def classify_columns(df):
    boolean_cols = df.select_dtypes(include=['bool']).columns.tolist()
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    string_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    console.print("[bold cyan]Clasificación de columnas:[/bold cyan]")
    console.print(f"[green]Booleanas:[/green] {boolean_cols}")
    console.print(f"[green]Numéricas:[/green] {numeric_cols}")
    console.print(f"[green]Cadenas de texto:[/green] {string_cols}")
    
    return boolean_cols, numeric_cols, string_cols

---
## funcion: `drop_columns_with_nulls(df, threshold=0.5)`
Elimina columnas con más del 50% de valores nulos para evitar ruido.

In [9]:
def drop_columns_with_nulls(df, threshold=0.5):
    null_percent = df.isnull().mean() * 100

    table = Table(title="Porcentaje de valores nulos por columna")
    table.add_column("Columna", style="cyan", no_wrap=True)
    table.add_column("Porcentaje", style="yellow")

    for col, val in null_percent.sort_values(ascending=False).items():
        table.add_row(str(col), f"{val:.2f}%")

    console.print(table)

    cols_to_drop = df.isnull().mean()[df.isnull().mean() > threshold].index.tolist()
    console.print(f"\n[bold red]Columnas eliminadas por alto porcentaje de nulos (>{threshold*100}%):[/bold red] {cols_to_drop}")

    df_clean = df.drop(columns=cols_to_drop)
    return df_clean


---
## funcion: `drop_rows_with_few_nulls(df, threshold=0.01)`
Elimina filas con valores nulos en columnas donde los nulos representan menos del 1% de las observaciones.

In [10]:
def drop_rows_with_few_nulls(df, threshold=0.01):
    null_percent = df.isnull().mean() * 100
    table = Table(title="Porcentaje de valores nulos por columna")
    table.add_column("Columna", style="cyan", no_wrap=True)
    table.add_column("Porcentaje", style="yellow")
    
    for col, val in null_percent.sort_values(ascending=False).items():
        table.add_row(str(col), f"{round(val)}%")
    
    console.print(table)
    
    cols_to_clean = null_percent[null_percent < threshold * 100].index.tolist()
    console.print(f"\n[bold green]Columnas con menos del {threshold*100}% de nulos:[/bold green] {cols_to_clean}")
    
    df_clean = df.dropna(subset=cols_to_clean)
    console.print("\n[bold red]Se eliminaron las filas que contenían nulos en las columnas mencionadas.[/bold red]")
    
    return df_clean


---
## funcion: `fill_nulls_in_column(df, column, strategy=None)`
Rellena los valores nulos de la columna especificada utilizando una estrategia de imputación que minimice el impacto en el modelo.

In [11]:
def fill_nulls_in_column(df, column, strategy=None):
    # Verificar que la columna exista
    if column not in df.columns:
        console.print(f"[bold red]La columna '{column}' no existe en el DataFrame.[/bold red]")
        return df

    # Mostrar información de los valores nulos en la columna
    total = len(df[column])
    missing = df[column].isnull().sum()
    percent_missing = (missing / total) * 100
    console.print(f"[bold cyan]La columna '{column}' tiene {missing} valores nulos de {total} ({percent_missing:.2f}%).[/bold cyan]")

    # Seleccionar estrategia por defecto según el tipo de dato, si no se especifica
    if strategy is None:
        if pd.api.types.is_numeric_dtype(df[column]):
            strategy = 'median'
        else:
            strategy = 'mode'
    
    # Aplicar la estrategia de imputación
    if strategy == 'median':
        if pd.api.types.is_numeric_dtype(df[column]):
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
            console.print(f"[bold green]Se han imputado los nulos de la columna '{column}' con la mediana: {median_value}[/bold green]")
        else:
            console.print(f"[bold red]La estrategia 'median' no es adecuada para la columna '{column}' de tipo {df[column].dtype}.[/bold red]")
    elif strategy == 'mean':
        if pd.api.types.is_numeric_dtype(df[column]):
            mean_value = df[column].mean()
            df[column] = df[column].fillna(mean_value)
            console.print(f"[bold green]Se han imputado los nulos de la columna '{column}' con la media: {mean_value}[/bold green]")
        else:
            console.print(f"[bold red]La estrategia 'mean' no es adecuada para la columna '{column}' de tipo {df[column].dtype}.[/bold red]")
    elif strategy == 'mode':
        mode_series = df[column].mode()
        if not mode_series.empty:
            mode_value = mode_series[0]
            df[column] = df[column].fillna(mode_value)
            console.print(f"[bold green]Se han imputado los nulos de la columna '{column}' con la moda: {mode_value}[/bold green]")
        else:
            console.print(f"[bold red]No se pudo calcular la moda para la columna '{column}'.[/bold red]")
    else:
        console.print(f"[bold red]La estrategia '{strategy}' no es reconocida. Use 'median', 'mean' o 'mode'.[/bold red]")
    
    console.print(f"[underline white]                               ")
    
    return df


---
## funcion: `convert_to_categorical(df)`
Reduce la cardinalidad en columnas “identifier” agrupando valores poco frecuentes.

In [12]:
from rich.console import Console
import pandas as pd

console = Console()

def convert_to_categorical(df):
    # Listamos las columnas originales para no iterar sobre columnas generadas por get_dummies.
    original_columns = df.columns.tolist()
    processed_cols = set()
    
    # Procesar columnas que contengan "identifier"
    for col in original_columns:
        if "identifier" in col.lower():
            threshold_value = len(df) * 0.05
            vc = df[col].value_counts()
            top_values = vc[vc > threshold_value].index
            # Definir un prefijo eliminando la palabra "Identifier" (manteniendo el resto) o usar uno por defecto.
            prefix = col.replace("Identifier", "").strip()
            prefix = f"{prefix}_" if prefix else "Identifier_"
            # Renombrar cada valor según si está en los top o no.
            df[col] = df[col].apply(lambda x: f"{prefix}{x}" if x in top_values else f"Otros{prefix.rstrip('_')}")
            # Aplicar One-Hot Encoding y eliminar la columna original.
            df = pd.get_dummies(df, columns=[col], prefix="", prefix_sep="")
            processed_cols.add(col)
            console.print(f"[bold green]Procesada columna '{col}' (basada en 'identifier') con One-Hot Encoding.[/bold green]")
    
    # Procesar columnas que contengan "version" y que no hayan sido procesadas ya
    for col in original_columns:
        if "version" in col.lower() and col not in processed_cols:
            threshold_value = len(df) * 0.05
            vc = df[col].value_counts()
            top_values = vc[vc > threshold_value].index
            # Definir un prefijo eliminando la palabra "Version" o usar uno por defecto.
            prefix = col.replace("Version", "").strip()
            prefix = f"{prefix}_" if prefix else "Version_"
            df[col] = df[col].apply(lambda x: f"{prefix}{x}" if x in top_values else f"Otros{prefix.rstrip('_')}")
            df = pd.get_dummies(df, columns=[col], prefix="", prefix_sep="")
            processed_cols.add(col)
            console.print(f"[bold green]Procesada columna '{col}' (basada en 'version') con One-Hot Encoding.[/bold green]")
    
    return df


---
## funcion: `drop_low_correlation_columns(df, threshold=0.01)`
Elimina las columnas numéricas (incluyendo las booleanas) que presentan una correlación muy baja con todas las demás columnas, según un umbral especificado.

In [13]:
def drop_low_correlation_columns(df, threshold=0.01):
    corr_matrix = df.corr()
    low_corr_cols = []
    
    for col in corr_matrix.columns:
        other_corr = corr_matrix[col].drop(labels=col)
        if (abs(other_corr) < threshold).all():
            low_corr_cols.append(col)
    
    df = df.drop(columns=low_corr_cols)
    console.print(f"[bold red]Columnas numéricas eliminadas por baja correlación (umbral {threshold}):[/bold red] {low_corr_cols}")
    return df


---
## funcion: `show_column_statistics(df, column)`
Muestra estadísticas descriptivas y visualizaciones básicas (histograma y boxplot) para la columna especificada.

In [14]:
def show_column_statistics(df, column):
    if column not in df.columns:
        console.print(f"[bold red]La columna '{column}' no existe en el DataFrame.[/bold red]")
        return

    console.print(f"[bold cyan]Estadísticas de la columna '{column}':[/bold cyan]")
    console.print(df[column].describe())
    
    # Histograma
    plt.figure(figsize=(10, 4))
    sns.histplot(df[column].dropna(), kde=True)
    plt.title(f"Histograma de '{column}'")
    plt.xlabel(column)
    plt.ylabel("Frecuencia")
    plt.show()

    # Boxplot
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[column])
    plt.title(f"Boxplot de '{column}'")
    plt.xlabel(column)
    plt.show()

---
## funcion: `plot_full_correlation_matrix(df)`
Genera y muestra la matriz de correlación de todas las columnas numéricas del DataFrame, ajustando dinámicamente el tamaño del gráfico según el número de variables para una visualización clara.

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_full_correlation_matrix(df):
    df_corr = df.copy()
    
    cat_cols = df_corr.select_dtypes(include=["category"]).columns
    if len(cat_cols) > 0:
        df_corr[cat_cols] = df_corr[cat_cols].apply(lambda s: s.cat.codes)
    
    bool_cols = df_corr.select_dtypes(include=["bool"]).columns
    if len(bool_cols) > 0:
        df_corr[bool_cols] = df_corr[bool_cols].astype(int)
    
    dt_cols = df_corr.select_dtypes(include=["datetime64[ns]"]).columns
    if len(dt_cols) > 0:
        df_corr[dt_cols] = df_corr[dt_cols].astype('int64')
    
    numeric_df = df_corr.select_dtypes(include=["number"])
    corr_matrix = numeric_df.corr()
    
    n = corr_matrix.shape[0]
    width = max(12, n)
    height = max(10, n)
    
    plt.figure(figsize=(width, height))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True)
    plt.title("Matriz de correlación completa")
    plt.show()


---
## Aplicacion de metodos

> Comenzaremos eliminando columnas en las que cada valor es único lo cual se trate de índices internos que no aportan información útil.

In [ ]:
df = drop_unique_columns(df)

---
> Ahora eliminamos columnas con más del 50% de valores nulos

In [ ]:
df = drop_columns_with_nulls(df, threshold=0.5)

---
> Conversion a categoricas?

In [ ]:
df = convert_to_categorical(df)

In [ ]:
df.head()

---
> Eliminar filas con pocos valores nulos


In [ ]:
df = drop_rows_with_few_nulls(df, threshold=0.1)

---
> Basados en el output de nuestra funcion anterior, vamos a aplicar nuestra funcion de imputacion de nulos `fill`
```
| SmartScreen                                       │ 36%        │
│ OrganizationIdentifier                            │ 31%        │
│ SMode                                             │ 6%         │
│ CityIdentifier                                    │ 4%         │
│ Wdft_IsGamer                                      │ 3%         │
│ Wdft_RegionIdentifier                             │ 3%         │
│ Census_InternalBatteryNumberOfCharges             │ 3%         │
│ Census_FirmwareManufacturerIdentifier             │ 2%         │
│ Census_FirmwareVersionIdentifier                  │ 2%         │
│ Census_IsFlightsDisabled                          │ 2%         │
│ Census_OEMModelIdentifier                         │ 1%         │
│ Census_OEMNameIdentifier                          │ 1%         │
│ Firewall                                          │ 1%         │
│ Census_TotalPhysicalRAM                           │ 1%         │
│ Census_IsAlwaysOnAlwaysConnectedCapable           │ 1%         │
│ Census_OSInstallLanguageIdentifier                │ 1%         │
│ IeVerIdentifier                                   │ 1%         │
│ Census_SystemVolumeTotalCapacity                  │ 1%         │
│ Census_PrimaryDiskTotalCapacity                   │ 1%         │
│ Census_InternalPrimaryDiagonalDisplaySizeInInches │ 1%         │
│ Census_InternalPrimaryDisplayResolutionVertical   │ 1%         │
│ Census_InternalPrimaryDisplayResolutionHorizontal │ 1%         |
```

In [ ]:
columnas_a_imputar = [
    "SmartScreen",
    "OrganizationIdentifier",
    "SMode",
    "CityIdentifier",
    "Wdft_IsGamer",
    "Wdft_RegionIdentifier",
    "Census_InternalBatteryNumberOfCharges",
    "Census_FirmwareManufacturerIdentifier",
    "Census_FirmwareVersionIdentifier",
    "Census_IsFlightsDisabled",
    "Census_OEMModelIdentifier",
    "Census_OEMNameIdentifier",
    "Firewall",
    "Census_TotalPhysicalRAM",
    "Census_IsAlwaysOnAlwaysConnectedCapable",
    "Census_OSInstallLanguageIdentifier",
    "IeVerIdentifier",
    "Census_SystemVolumeTotalCapacity",
    "Census_PrimaryDiskTotalCapacity",
    "Census_InternalPrimaryDiagonalDisplaySizeInInches",
    "Census_InternalPrimaryDisplayResolutionVertical",
    "Census_InternalPrimaryDisplayResolutionHorizontal"
]

for col in columnas_a_imputar:
    df = fill_nulls_in_column(df, col)


> Ahora ejecutaremos el funcion que nos ayudo a sacar los nulos por columna y asi ver como nos ha quedado:

In [ ]:
df = drop_rows_with_few_nulls(df, threshold=0.1)

Podemos ver que ahora no tenemos nulos en ninguna columna. Solo por si acaso ejecutaremos mas codigo para asegurarnos.

In [ ]:
df.isnull().mean().sort_values(ascending=False) * 100

> Como podemos ver ya no tenemos mas valores nulos.

In [24]:
def aplicar_ohe(df):
    string_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    if not string_cols:
        print("No hay columnas categóricas para codificar.")
        return df

    # Aplicar One-Hot Encoding
    df_ohe = pd.get_dummies(df, columns=string_cols, drop_first=True)
    
    return df_ohe

In [25]:
def normalizar_dataframe(df, metodo="minmax"):
    # Identificar columnas numéricas
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    
    if not numeric_cols:
        print("No hay columnas numéricas para normalizar.")
        return df  # Devuelve el DataFrame sin cambios si no hay columnas numéricas
    
    df_normalizado = df.copy()  # Copia para no modificar el original
    
    if metodo == "minmax":
        scaler = MinMaxScaler()
    elif metodo == "zscore":
        scaler = StandardScaler()
    else:
        raise ValueError("Método no válido. Usa 'minmax' o 'zscore'.")

    # Aplicar normalización a las columnas numéricas
    df_normalizado[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    return df_normalizado


In [26]:
df = aplicar_ohe(df)

In [27]:
df = normalizar_dataframe(df)

In [ ]:
df.info()

In [ ]:
df = drop_low_correlation_columns(df, threshold=0.01)

In [ ]:
df.info()

In [29]:
# plot_full_correlation_matrix(df)